# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import pandas as pd
pd.set_option('display.max_columns', 100)
import os
import re
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Response_disaster.db')
df = pd.read_sql_table('Response_disaster_table', engine)
df

,id,message,original,genre,categories,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,related-0;request-0;offer-0;aid_related-0;medi...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,related-1;request-1;offer-0;aid_related-1;medi...,1,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,related-0;request-0;offer-0;aid_related-0;medi...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,related-1;request-1;offer-0;aid_related-1;medi...,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [3]:
df.shape

(26216, 41)

In [4]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
categories                    0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_cent

In [5]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
# drop 'child_alone' column as it has only value of zero
df = df.drop('child_alone', axis = 1)

In [7]:
#min,max value for each columns
print(df.min())
print(df.max())

id                                                                        2
message                                                                    
genre                                                                direct
categories                related-0;request-0;offer-0;aid_related-0;medi...
related                                                                   0
request                                                                   0
offer                                                                     0
aid_related                                                               0
medical_help                                                              0
medical_products                                                          0
search_and_rescue                                                         0
security                                                                  0
military                                                                  0
water       

In [8]:
#Replace "related" value" with '0' or '1'. For the value of 2, Replace it with '1'
df['related'] = df['related'].map(lambda x: 1 if x==2 else x)     

In [9]:
# Define X and Y
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre','categories'], axis = 1)

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [11]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
def ML_pipeline(clf = AdaBoostClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline

# include StartingVerbExtractor custom transformer
def ML_pipeline_2(clf = AdaBoostClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
#data Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y)
#Pipeline Creation
model = ML_pipeline()
#Model Training
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
y_pred_test = model.predict(X_test)

# on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      4991
               request       0.80      0.53      0.64      1116
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.61      0.68      2699
          medical_help       0.60      0.29      0.39       512
      medical_products       0.65      0.29      0.40       343
     search_and_rescue       0.55      0.18      0.27       167
              security       0.24      0.06      0.10       117
              military       0.68      0.34      0.45       188
                 water       0.73      0.64      0.68       426
                  food       0.80      0.68      0.74       710
               shelter       0.74      0.53      0.62       573
              clothing       0.73      0.33      0.45        97
                 money       0.58      0.28      0.38       150
        missing_people       0.54      

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
# get list of model parameters
model.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'features__text_pipeline__vect__tokenizer', 

In [17]:
# Initialize the machine learning pipeline
model_2 = ML_pipeline()
# Define the hyperparameter grid
parameters = {
    'clf__estimator__learning_rate': [0.5, 1.0],
    'clf__estimator__n_estimators': [10, 20]
}
# Set up GridSearchCV
cv = GridSearchCV(model_2, param_grid=parameters, cv=5, n_jobs=-1, verbose=3) 
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estima

[Parallel(n_jobs=-1)]: Done   3 out of  20 | elapsed: 12.7min remaining: 72.1min


[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20 
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.21408593948639715, total= 9.9min
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.20396744659206512, total= 9.9min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.21770091556459817, total= 9.9min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.21891685736079328, total= 9.9min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.20798372743452834, total=10.0min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.245676500508647, total=10.0min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.23296032553407936, total=10.0min


[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed: 13.0min remaining: 13.0min


[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.25349605898804983, total=14.4min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.21668362156663276, total=14.4min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.24338758901322483, total=14.4min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.22634791454730416, total=14.4min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.23010424612255276, total=14.7min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.2250190694126621, total=14.9min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.25076297049847407, total= 4.8min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.25940996948118006, total= 4.9min


[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed: 18.1min remaining:  3.2min


[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.2425629290617849, total= 4.9min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.2321973550356053, total= 4.9min


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 18.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__learning_rate': [0.5, 1.0], 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
y_pred_test = cv.predict(X_test)

# on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.96      0.88      4991
               request       0.77      0.51      0.61      1116
                 offer       0.00      0.00      0.00        34
           aid_related       0.74      0.56      0.64      2699
          medical_help       0.61      0.27      0.37       512
      medical_products       0.62      0.26      0.37       343
     search_and_rescue       0.62      0.23      0.33       167
              security       0.29      0.05      0.09       117
              military       0.63      0.23      0.34       188
                 water       0.72      0.64      0.68       426
                  food       0.80      0.66      0.72       710
               shelter       0.78      0.52      0.62       573
              clothing       0.75      0.46      0.57        97
                 money       0.59      0.30      0.40       150
        missing_people       0.57      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
# try other model -> RandomForestClassifier
rf_model = ML_pipeline(clf = RandomForestClassifier())
rf_model.fit(X_train, y_train)
y_pred_rf_test = rf_model.predict(X_test)

print(classification_report(y_test.values, y_pred_rf_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.94      0.88      4991
               request       0.84      0.39      0.53      1116
                 offer       0.00      0.00      0.00        34
           aid_related       0.74      0.52      0.61      2699
          medical_help       0.64      0.09      0.15       512
      medical_products       0.78      0.07      0.13       343
     search_and_rescue       0.76      0.08      0.14       167
              security       0.00      0.00      0.00       117
              military       0.60      0.10      0.17       188
                 water       0.83      0.20      0.32       426
                  food       0.83      0.42      0.56       710
               shelter       0.78      0.15      0.26       573
              clothing       0.91      0.10      0.19        97
                 money       0.33      0.01      0.01       150
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# Using ML_pipeline_2 that includes custom transformer of 'StartingVerbEstimator'
model_3 = ML_pipeline_2()
model_3.fit(X_train, y_train)
y_pred_3_test = model_3.predict(X_test)

print(classification_report(y_test.values, y_pred_3_test, target_names=y.columns.values))

### 9. Export your model as a pickle file

In [ ]:
# save model in pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(model_3, f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.